In [29]:
truth = [1, 2, 3, 4, 5, 6]
prediction = [1, 3, 5, 6]

truth_pairs = [(truth[j], truth[j+1]) for j in range(0, len(truth)-1)]
prediction_pairs = [(prediction[j], prediction[j+1]) for j in range(0, len(prediction)-1)]

sum(correct_choice in prediction_pairs for correct_choice in truth_pairs) / len(truth_pairs)

0.2